# Modele analizy danych

### 2024/2025, semestr zimowy
Tomasz Rodak

---

## Całkowanie metodą Monte Carlo

Metoda Monte Carlo to grupa algorytmów probabilistycznych pozwalających na uzyskanie przybliżonych wyników w sytuacjach, w których dokładne rozwiązanie jest trudne lub niemożliwe do uzyskania. Metoda ta polega na estymacji poszukiwanej numerycznej wartości poprzez generowanie losowych próbek z rozkładu związanego z problemem. 

### Wprowadzenie. Obliczanie miary zbioru

Zastosujemy metodę Monte Carlo do obliczenia pola powierzchni koła o promieniu 1. Załóżmy, że mamy koło o promieniu 1:

$$x^2 + y^2 <= 1$$

wpisane w kwadrat o boku 2:

$$-1 <= x <= 1$$
$$-1 <= y <= 1$$

Losujemy $N$ punktów z kwadratu i sprawdzamy, ile z nich znajduje się w kole:

In [ ]:
import numpy as np

N = 10_000 # liczba prób
points = np.random.uniform(-1, 1, (N, 2)) # generowanie punktów
inside = points[:, 0]**2 + points[:, 1]**2 <= 1 # punkty wewnątrz koła
n_inside = np.sum(inside) # liczba punktów wewnątrz koła
n_inside

Rysunek:

In [ ]:
from matplotlib import pyplot as plt
plt.style.use('ggplot') # styl wykresu; można zmienić na inny lub usunąć

# kwadrat 
plt.plot([-1, -1, 1, 1, -1], [-1, 1, 1, -1, -1], 'k-')
# koło
t = np.linspace(0, 2*np.pi, 1000) # 1000 równo rozmieszczonych punktów na przedziale [0, 2*pi]
plt.plot(np.cos(t), np.sin(t), 'k-')
# punkty wewnątrz koła, czerwone
plt.plot(points[inside, 0], points[inside, 1], 'ro', markersize=1)
# punkty na zewnątrz koła, niebieskie
plt.plot(points[~inside, 0], points[~inside, 1], 'bo', markersize=1)
# uswawienie osi
plt.axis('equal');

Zdroworozsądkowe myślenie podpowiada, że stosunek liczby punktów w kole do liczby wszystkich wylosowanych punktów (czyli $N$)
 aproksymuje stosunek pól koła i kwadratu:

$$\frac{\text{liczba punktów w kole}}{N} \approx \frac{\text{pole koła}}{\text{pole kwadratu}}
=\frac{\text{pole koła}}{4}$$

i że przybliżenie to powinno być tym lepsze, im większa jest liczba losowanych punktów $N$.

Pole koła aproksymowane przez nasz eksperyment to:

In [ ]:
est_area = 4 * n_inside / N 
est_area

Prawdziwy wynik to:

$$
\pi r^2 = \pi,
$$

bo $r=1$. Niechcący aproksymujemy więc wartość liczby $\pi$ :-)

Błąd względny:

In [ ]:
relative_error = np.abs(np.pi - est_area) / np.pi
print(f'Błąd względny: {relative_error * 100:g}%')

### Uzasadnienie metody

Skuteczność metody Monte Carlo wynika z prawa wielkich liczb i następującego triku rachunkowego.

Załóżmy, że chcemy obliczyć całkę z funkcji $f(\mathbf{x})$ określonej na obszarze ograniczonym $\Omega\subset\mathbb{R}^n$. Wówczas całkę tę możemy zapisać jako wartość oczekiwaną zmiennej losowej $f(\mathbf{x})$ względem rozkładu jednostajnego na zbiorze $\Omega$:

\begin{equation*}
\begin{aligned}
\int_\Omega f(\mathbf{x}) d\mathbf{x} &= \text{Vol}(\Omega) \int_\Omega f(\mathbf{x}) \frac{d\mathbf{x}}{\text{Vol}(\Omega)} \\
&= \text{Vol}(\Omega) \mathbb{E}[f(\mathbf{x})].
\end{aligned}
\end{equation*} 

Ostatnia równość wynika z faktu, że funkcja

\begin{equation*}
p(\mathbf{x}) = \frac{1}{\text{Vol}(\Omega)}, \quad \mathbf{x}\in\Omega
\end{equation*}

jest gęstością rozkładu jednostajnego na zbiorze $\Omega$ i ze wzoru na wartość oczekiwaną ciągłej zmiennej losowej:

\begin{equation*}
\mathbb{E}[f(\mathbf{x})] = \int_\Omega f(\mathbf{x}) p(\mathbf{x}) d\mathbf{x}.
\end{equation*}

Z prawa wielkich liczb wynika, że

\begin{equation*}
\frac{1}{N} \sum_{i=1}^N f(\mathbf{x}_i) \xrightarrow{} \mathbb{E}[f(\mathbf{x})],\quad N\to\infty,
\end{equation*}

gdzie $\mathbf{x}_i$ są niezależnymi próbkami z rozkładu $p(\mathbf{x})$. Zatem

\begin{equation*}
\frac{\text{Vol}(\Omega)}{N} \sum_{i=1}^N f(\mathbf{x}_i) \xrightarrow{} \int_\Omega f(\mathbf{x}) d\mathbf{x},\quad N\to\infty.
\end{equation*}

W naszym przykładzie zbiór $\Omega$ to kwadrat $[-1,1]\times[-1,1]$, a jako funkcję $f(\mathbf{x})$ możemy wziąć funkcję charakterystyczną koła o promieniu 1:

\begin{equation*}
f(\mathbf{x}) = \mathbb{1}_{\{x^2 + y^2 \leq 1\}}(\mathbf{x})
 = \begin{cases} 1, & \text{jeśli } x^2 + y^2 \leq 1, \\ 0, & \text{w przeciwnym przypadku.} \end{cases}
\end{equation*}

Zatem

\begin{equation*}
\begin{aligned}
\text{pole koła} &= \int_{[-1,1]\times[-1,1]} \mathbb{1}_{\{x^2 + y^2 \leq 1\}}(\mathbf{x}) d\mathbf{x}\\
&\approx \frac{4}{N} \sum_{i=1}^N \mathbb{1}_{\{x^2 + y^2 \leq 1\}}(x_i, y_i)\\
&= \frac{4}{N} \sum_{i=1}^N \begin{cases} 1, & \text{jeśli } x_i^2 + y_i^2 \leq 1, \\ 0, & \text{w przeciwnym przypadku} \end{cases}\\
&= \frac{4}{N}\cdot \text{liczba punktów w kole}.
\end{aligned}
\end{equation*}


**Uwaga:** Aby stosować metodę Monte Carlo do obliczenia całki, musimy uprzednio wiedzieć, że całka ta istnieje i jest skończona. 

### Zadanie 1.1

Oblicz metodą Monte Carlo objętość $n$-wymiarowej kuli o promieniu 1. Ustal $N=10^6$ i zobacz, jak zachowuje się błąd względny w zależności od wymiaru $n$.

### Zadanie 1.2

Oblicz metodą Monte Carlo pole powierzchni pod wykresem funkcji $f(x) = x^2$ na przedziale $[0,1]$. Porównaj wynik z wartością dokładną.

### Zadanie 1.3

Napisz funkcję `mc_integrate(f, a, b, N=10_000)`, która oblicza przybliżoną wartość całki $\int_a^b f(x) dx$ metodą Monte Carlo. Funkcja ma przyjmować jako argumenty:
* funkcję $f$ - obiekt wywoływalny (np. funkcja, lambda), reprezentujący funkcję, której całkę chcemy obliczyć,
* liczby $a$, $b$ - przedział całkowania,
* liczbę całkowitą $N$ o domyślnej wartości 10_000 - liczbę losowanych punktów.

Przetestuj działanie funkcji na przykładach.

### Zadanie 1.4

Wykonaj obliczenia z zadania 1.2 dla $N=10^3, 10^4, 10^5, 10^6, 10^7$. Jak zmienia się błąd względny wraz ze wzrostem liczby próbek $N$?

### Zadanie 1.5

Wykonaj obliczenia z zadania 1.2 po 100 razy dla każdego $N=10^2,10^3,10^4,10^5,10^6$. Zapamiętaj otrzymane wartości w tablicy. 

1. Narysuj wykresy pudełkowe (boxplot) otrzymanych wyników dla każdego $N$. Jak zmienia się rozrzut aproksymacji wraz ze wzrostem $N$?
2. O ile musimy zwiększyć $N$, aby odchylenie standardowe wyników zmniejszyło się dwukrotnie? Sprawdź swoje przypuszczenie dla $N=10^4$.